# Bayesian SIR Model with Change Points

In [1]:
import numpy as np
from scipy.stats import binom, gamma, beta, expon, poisson, uniform, bernoulli
from joblib import Parallel, delayed
from scipy.special import gamma as gammaFunc
import random

import pandas as pd

import matplotlib.pyplot as plt

import tqdm

import itertools

## Data Simulation

In [2]:
def sim_dataset(chg_pt, scenarios, T, S0, I0, R0, n_datasets):

    N = S0 + I0 + R0
    n_sc = scenarios.shape[0]

    # create array of transmission and removal rate parameters at each time step
    beta  = np.array([[scenarios[i,(chg_pt <= t+1).sum(),0] for t in range(T)] for i in range(n_sc)])
    gamma = np.array([[scenarios[i,(chg_pt <= t+1).sum(),1] for t in range(T)] for i in range(n_sc)])


    Delta_I = np.zeros(shape=(n_datasets, n_sc, T), dtype=np.int32)
    Delta_R = np.zeros(shape=(n_datasets, n_sc, T), dtype=np.int32)
    S       = np.zeros(shape=(n_datasets, n_sc, T), dtype=np.int32)
    I       = np.zeros(shape=(n_datasets, n_sc, T), dtype=np.int32)
    R       = np.zeros(shape=(n_datasets, n_sc, T), dtype=np.int32)

    Delta_I[:,:,0] = binom.rvs(S0, 1-np.exp(-beta[:,0]*I0/N), size=(n_datasets, n_sc))
    Delta_R[:,:,0] = binom.rvs(I0, gamma[:,0], size=(n_datasets, n_sc))
    S[:,:,0]       = S0 - Delta_I[:,:,0]
    I[:,:,0]       = I0 + Delta_I[:,:,0] - Delta_R[:,:,0]
    R[:,:,0]       = R0 + Delta_R[:,:,0]

    for t in range(1, T):
        Delta_I[:,:,t] = binom.rvs(S[:,:,t-1], 1-np.exp(-beta[:,t]*I[:,:,t-1]/N))
        Delta_R[:,:,t] = binom.rvs(I[:,:,t-1], gamma[:,t])
        S[:,:,t]       = S[:,:,t-1] - Delta_I[:,:,t]
        I[:,:,t]       = I[:,:,t-1] + Delta_I[:,:,t] - Delta_R[:,:,t]
        R[:,:,t]       = R[:,:,t-1] + Delta_R[:,:,t]

    return Delta_I, Delta_R, S, I, R

In [3]:
def plot_SRI(S, I, R, sc=0, d=None, start_cond=(999_950, 50, 0), tot=1_000_000, time=100):

    if d is None:
        S = np.expand_dims(np.mean(S, axis=0), 0)
        I = np.expand_dims(np.mean(I, axis=0), 0)
        R = np.expand_dims(np.mean(R, axis=0), 0)
        d = 0

    S = np.concatenate([[start_cond[0]], S[d,sc]])
    I = np.concatenate([[start_cond[1]], I[d,sc]])
    R = np.concatenate([[start_cond[2]], R[d,sc]])

    # plot
    fig, ax = plt.subplots()

    y = np.vstack([S, I, R])
    ax.stackplot(np.arange(time+1), y/tot, labels=["S","I","R"], alpha=0.8)

    ax.set_xlabel("Day")
    ax.set_ylabel("Proportion")
    ax.set_xticks(np.concatenate([[0], np.arange(25, time+1, 25)]))
    ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    plt.legend(loc="upper right")

    plt.show()

### Example

In [4]:
T = 100
N = 1_000_000

chg_pt = np.array([26, 51, 76])

sc_1 = [(0.3, 0.05), (0.4, 0.15), (0.25, 0.2),  (0.2,  0.25)]
sc_2 = [(0.4, 0.1),  (0.4, 0.25), (0.25, 0.25), (0.25, 0.4) ]
sc_3 = [(0.5, 0.1),  (0.3, 0.3),  (0.4,  0.2),  (0.2,  0.4) ]
scenarios = np.array([sc_1, sc_2, sc_3])

S0 = N-50
I0 = 50
R0 = 0

n_datasets = 100

In [5]:
Delta_I, Delta_R, S, I, R =  sim_dataset(chg_pt, scenarios, T, S0, I0, R0, n_datasets)
#for i in range(len(scenarios)):
 #   print(f"Scenario {i+1}:")
#    plot_SRI(S, I, R, sc=i)

## Gibbs Sampling

In [7]:
S_obs = np.concatenate(([S0],S[0,0]))
I_obs = np.concatenate(([I0],I[0,0]))
R_obs = np.concatenate(([R0],R[0,0]))
PI_obs = I_obs/N
N_infect_obs = np.concatenate(([0],Delta_I[0,0]))
N_recovery_obs = np.concatenate(([0],Delta_R[0,0]))

data = pd.DataFrame({
        'susceptible': S_obs,
        'infects': I_obs,
        'recovered': R_obs,
        'PI': PI_obs,
        'deltaI': N_infect_obs,
        'deltaR': N_recovery_obs
    })

In [8]:
data

,susceptible,infects,recovered,PI,deltaI,deltaR
0,999950,50,0,0.000050,0,0
1,999932,62,6,0.000062,18,6
2,999913,77,10,0.000077,19,4
3,999883,104,13,0.000104,30,3
4,999854,127,19,0.000127,29,6
...,...,...,...,...,...,...
96,164982,18,835000,0.000018,1,4
97,164981,12,835007,0.000012,1,7
98,164981,8,835011,0.000008,0,4
99,164981,6,835013,0.000006,0,2


In [18]:
def calculate_contact_hat_parallel(pp_lambda_t):
    pp, lambda_t = pp_lambda_t
    return np.sum(poisson.ppf(pp, lambda_t))

def gibbs_sampling(data, samples=10000, T_max=100, burnin=1000, thinning=10):
    
    ############# data
    # Data is expected to be a dataframe with 101 rows (100 steps + initial one)
    # OBS: the initial row should display the starting values 
    #      for Susceptible (S), Infected (I), and Recovered (R), with zero values in the 
    #      columns representing changes (deltas) over time.
    
    I_obs = data["infects"].values
    S_obs = data["susceptible"].values
    PI_obs = data["PI"].values
    N_infect_obs = data["deltaI"].values[1:]
    N_recovery_obs = data["deltaR"].values[1:]
    

    #######################
    ##  HYPERPARAMETERS  ##
    #######################
    
    p_a = 1/T_max**4
    p_b = 2 - p_a
    
    b_shape = 0.1
    b_rate = 0.1
    r_shape = 0.1
    r_rate = 0.1
    
    gamma_b_shape = gammaFunc(b_shape)
    gamma_r_shape = gammaFunc(r_shape)
    

    ######################
    ##  INITIALIZATION  ##
    ######################
    
    ##### delta
    Delta_hat = np.zeros(shape=T_max, dtype=int)
    Delta_hat[0] = 1
    Stage_hat = np.cumsum(Delta_hat, dtype=int)-1
    K_hat     = np.sum(Delta_hat, dtype=int)

    ##### b and r
    b_hat    = gamma.rvs(a=b_shape, scale=1/b_rate, size=K_hat)
    r_hat    = gamma.rvs(a=r_shape, scale=1/r_rate, size=K_hat)
    
    ##### beta and gamma
    beta_hat    = expon.rvs(scale=1/b_hat[Stage_hat[0]], size=T_max)
    gamma_hat   = beta.rvs(a=r_hat[Stage_hat[0]], b=1, size=T_max)
    lambda_t    = beta_hat*PI_obs[:-1]
    p_upper     = 1 - poisson.cdf(0, lambda_t)
    pp          = [uniform.rvs(size=N_infect_obs[t]) * p_upper[t] + (1 - p_upper[t]) for t in range(T_max)]
    contact_hat = np.array(Parallel(n_jobs=-1)(delayed(calculate_contact_hat_parallel)((pp[t], lambda_t[t],)) for t in range(T_max)))
    
    
    beta_hat    = gamma.rvs(a=contact_hat + 1, scale=1/(b_hat[Stage_hat] + PI_obs[:-1]*S_obs[:-1]))
    gamma_hat   = beta.rvs(a=N_recovery_obs + r_hat[Stage_hat], b=1 + I_obs[:-1] - N_recovery_obs)
    lambda_t    = beta_hat * PI_obs[:-1]
    p_upper     = 1 - poisson.cdf(0, lambda_t)
    pp          = [uniform.rvs(size=N_infect_obs[t]) * p_upper[t] + (1 - p_upper[t]) for t in range(T_max)]
    contact_hat = np.array(Parallel(n_jobs=-1)(delayed(calculate_contact_hat_parallel)((pp[t], lambda_t[t],)) for t in range(T_max)))
    
    
    print("Initialization:\n")
    print("Delta_hat:",Delta_hat)
    print("b_hat:",b_hat)
    print("r_hat:",r_hat)
    print("beta_hat:",beta_hat)
    print("gamma_hat:",gamma_hat)
    print("----------------------------------------------------")
    print("----------------------------------------------------")
    

    ################
    ##  SAMPLING  ##
    ################
    
    Delta_all = []
    Stage_all = []
    b_all     = []
    r_all     = []
    beta_all  = []
    gamma_all = []
    
    for step in tqdm.tqdm(range(samples)):
        
        ###### delta_hat sampling
        ## add–delete–swap - proposal step
        # -1 delete
        # 0 swap
        # +1 add
                
        if K_hat==1:
            change_type = 1
        
        elif K_hat==T_max:
            change_type = -1

        else:
            change_type = np.random.choice([-1, 0, 1])
            
        Delta_hat_candidate = Delta_hat.copy()
        
        if change_type != 0:
            
            if change_type == 1:
                possible_change_indices = np.where(Delta_hat[1:] == 0)[0]+1
            elif change_type == -1:
                possible_change_indices = np.where(Delta_hat[1:] == 1)[0]+1
            index_to_change = np.random.choice(possible_change_indices)

            Delta_hat_candidate[index_to_change] = 1 - Delta_hat_candidate[index_to_change]
            Stage_hat_candidate = np.cumsum(Delta_hat_candidate)-1

            if change_type == 1:
                possible_change_indices_candidate = np.where(Delta_hat_candidate[1:] == 1)[0] + 1
                phase_original  = np.array([Stage_hat[index_to_change]])
                phase_candidate = np.array([Stage_hat[index_to_change], Stage_hat_candidate[index_to_change]])
            elif change_type == -1:
                possible_change_indices_candidate = np.where(Delta_hat_candidate[1:] == 0)[0] + 1
                phase_original  = np.array([Stage_hat[index_to_change], Stage_hat_candidate[index_to_change]])
                phase_candidate = np.array([Stage_hat_candidate[index_to_change]])
        
            logp_original = 0
            logp_candidate = 0

            for i in phase_original:
                L_i_original = np.where(Stage_hat == i)[0]

                logp_original += np.sum(np.log(np.arange(1, len(L_i_original)) - 1 + b_shape)) \
                               - np.log(gamma_b_shape) + b_shape * np.log(b_rate) \
                               - (b_shape + len(L_i_original)) * np.log(b_rate + np.sum(beta_hat[L_i_original]))
                logp_original += np.sum(np.log(np.arange(1, len(L_i_original)) - 1 + r_shape)) \
                               - np.log(gamma_r_shape) + r_shape * np.log(r_rate) \
                               - (r_shape + len(L_i_original)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_original])))
                logp_original -= np.sum(np.log(np.arange(1, len(L_i_original))))

            for i in phase_candidate:
                L_i_candidate = np.where(Stage_hat_candidate == i)[0]

                logp_candidate += np.sum(np.log(np.arange(1, len(L_i_candidate)) - 1 + b_shape)) \
                                - np.log(gamma_b_shape) + b_shape * np.log(b_rate) \
                                - (b_shape + len(L_i_candidate)) * np.log(b_rate + np.sum(beta_hat[L_i_candidate]))
                logp_candidate += np.sum(np.log(np.arange(1, len(L_i_candidate)) - 1 + r_shape)) \
                                - np.log(gamma_r_shape) + r_shape * np.log(r_rate) \
                                - (r_shape + len(L_i_candidate)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_candidate])))
                logp_candidate -= np.sum(np.log(np.arange(1, len(L_i_candidate))))

            logp_candidate += np.log(p_a / p_b) + np.log((3 - 2 * (K_hat == 1)) \
                            * len(possible_change_indices) / (3 - 2 * ((K_hat + 1) == T_max)) / len(possible_change_indices_candidate))

            ratio = np.exp(min([0, logp_candidate - logp_original]))

        elif change_type == 0:
            
            possible_change_indices = np.where(np.abs(Delta_hat[1:-1] - Delta_hat[2:]) == 1)[0]+1
            index_to_change = np.random.choice(possible_change_indices, 1)

            Delta_hat_candidate[index_to_change + np.array([0, 1])] = Delta_hat_candidate[index_to_change + np.array([1, 0])]
            Stage_hat_candidate = np.cumsum(Delta_hat_candidate)-1
   
            possible_change_indices_candidate = np.where(np.abs(Delta_hat_candidate[1:-1] - Delta_hat_candidate[2:]) == 1)[0]+1
        
            phase = np.array([Stage_hat[index_to_change], Stage_hat_candidate[index_to_change]])
    
            logp_original = 0
            logp_candidate = 0
        
            for i in phase:
                L_i_original = np.where(Stage_hat == i)[0]
                logp_original += np.sum(np.log(np.arange(1, len(L_i_original)) - 1 + b_shape)) \
                               - np.log(gamma_b_shape) + b_shape * np.log(b_rate) \
                               - (b_shape + len(L_i_original)) * np.log(b_rate + np.sum(beta_hat[L_i_original]))
                logp_original += np.sum(np.log(np.arange(1, len(L_i_original)) - 1 + r_shape)) \
                               - np.log(gamma_b_shape) + r_shape * np.log(r_rate) \
                               - (r_shape + len(L_i_original)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_original])))
                logp_original -= np.sum(np.log(np.arange(1, len(L_i_original))))

                L_i_candidate = np.where(Stage_hat_candidate == i)[0]
                logp_candidate += np.sum(np.log(np.arange(1, len(L_i_candidate)) - 1 + b_shape)) \
                                - np.log(gamma_b_shape) + b_shape * np.log(b_rate) \
                                - (b_shape + len(L_i_candidate)) * np.log(b_rate + np.sum(beta_hat[L_i_candidate]))
                logp_candidate += np.sum(np.log(np.arange(1, len(L_i_candidate)) - 1 + r_shape)) \
                                - np.log(gamma_b_shape) + r_shape * np.log(r_rate) \
                                - (r_shape + len(L_i_candidate)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_candidate])))
                logp_candidate -= np.sum(np.log(np.arange(1, len(L_i_candidate))))

            ratio = np.exp(min([0, logp_candidate - logp_original + np.log(len(possible_change_indices) / len(possible_change_indices_candidate))]))
        
        cxx = np.random.binomial(1, ratio)      
        if cxx == 1:
            Delta_hat = Delta_hat_candidate
            Stage_hat = Stage_hat_candidate
        
        
        ##### b and r sampling
            
        K_hat = np.sum(Delta_hat)
        
        b_hat = np.zeros(K_hat)
        r_hat = np.zeros(K_hat)
        for i in range(K_hat):
            L_i = np.where(Stage_hat == i)[0]
            b_hat[i] = gamma.rvs(size=1, a=(b_shape + len(L_i)), scale=1/(b_rate + np.sum(beta_hat[L_i])))
            r_hat[i] = gamma.rvs(size=1, a=(r_shape + len(L_i)), scale=1/(r_rate + np.sum(-np.log(gamma_hat[L_i]))))
    
        
        ##### beta and gamma sampling

        beta_hat    = gamma.rvs(a=contact_hat + 1, scale=1/(b_hat[Stage_hat] + PI_obs[:-1]*S_obs[:-1]))
        gamma_hat   = beta.rvs(a=N_recovery_obs + r_hat[Stage_hat], b=1 + I_obs[:-1] - N_recovery_obs)
        lambda_t    = beta_hat * PI_obs[:-1]
        p_upper     = 1 - poisson.cdf(0, lambda_t)
        pp          = [uniform.rvs(size=N_infect_obs[t]) * p_upper[t] + (1 - p_upper[t]) for t in range(T_max)]
        contact_hat = np.array(Parallel(n_jobs=-1)(delayed(calculate_contact_hat_parallel)((pp[t], lambda_t[t],)) for t in range(T_max)))
    
       


        if step % 100 == 0 and step != 0:
            print("\nStep:",step)
            print("Delta_hat:",Delta_hat)
            print("b_hat:",b_hat)
            print("r_hat:",r_hat)
            print("beta_hat:",beta_hat)
            print("gamma_hat:",gamma_hat)
            print("----------------------------------------------------")
            
        Delta_all.append(Delta_hat)
        Stage_all.append(Stage_hat)
        b_all.append(b_hat)
        r_all.append(r_hat)
        beta_all.append(beta_hat)
        gamma_all.append(gamma_hat)
  
    # Creazione del DataFrame
    MCMC_chain = {
        'Delta': Delta_all[burnin::thinning],
        'Stage': Stage_all[burnin::thinning],
        'b': b_all[burnin::thinning],
        'r': r_all[burnin::thinning],
        'beta': beta_all[burnin::thinning],
        'gamma': gamma_all[burnin::thinning]
            }
    
    return MCMC_chain

In [19]:
MCMC_chain=gibbs_sampling(data)

Initialization:

Delta_hat: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
b_hat: [4.09796111e-06]
r_hat: [0.57164757]
beta_hat: [4.53894700e+00 4.44961330e+00 3.81016348e+00 2.04393158e+00
 2.50974185e+00 8.24534661e+00 9.12106344e+00 1.79874813e+01
 1.30002900e+02 1.45898399e+00 1.21133519e+01 9.30572041e+00
 2.88502024e+01 1.20244059e+01 1.62183682e+01 3.14332748e+00
 1.33622694e+02 2.11544587e+02 1.31187710e+02 1.53326259e+02
 1.68113051e+02 1.57989159e+02 4.89696334e+02 6.23844230e+01
 1.16080529e+03 2.55617653e+02 2.72915991e+02 5.61832766e+03
 1.67117677e+03 7.65609664e+03 1.41237177e+04 1.12681913e+04
 2.45436558e+02 4.33151140e+03 5.62608232e+03 7.47610848e+03
 1.10815366e+04 1.62812009e+04 1.38568847e+04 3.17177169e+04
 8.45404013e+02 4.65997374e+04 3.14150853e+03 3.11266096e+04
 6.64168728e+02 3.03118501e+04 8.31363452e+

  1%|▍                                         | 101/10000 [01:14<2:04:00,  1.33it/s]


Step: 100
Delta_hat: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
b_hat: [4.05429031 2.66699519 2.36139748 5.244219   7.37284401]
r_hat: [0.32462958 0.46042846 0.71252197 0.6940179  0.63681543]
beta_hat: [0.23281791 0.30718708 0.43577078 0.21577516 0.26985523 0.29555494
 0.30195047 0.25135591 0.25872365 0.33044275 0.30041983 0.26055513
 0.29140199 0.24628069 0.30712944 0.30735815 0.29610876 0.32889032
 0.31804141 0.2909921  0.2894966  0.29838411 0.28210813 0.30983087
 0.30042223 0.39623171 0.40365548 0.39478937 0.40331754 0.40088718
 0.39250367 0.40351315 0.39667837 0.40181638 0.40227828 0.40550319
 0.39796807 0.3951711  0.39980816 0.40349483 0.39861442 0.39911217
 0.40023646 0.40398665 0.39915171 0.39779073 0.3989871  0.39868888
 0.39939521 0.40084155 0.25094412 0.25055145 0.24486397 0.25161222
 0.24527113 0.24497039 0.25302772

  2%|▊                                         | 201/10000 [02:32<2:08:22,  1.27it/s]


Step: 200
Delta_hat: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
b_hat: [3.50285159 2.42359582 3.73726541 5.43391384 5.58853891]
r_hat: [0.43725643 0.46911448 0.3227328  0.69490316 0.67290422]
beta_hat: [0.46855699 0.32769367 0.36006212 0.38654386 0.29203243 0.32696272
 0.38796453 0.20519786 0.27774044 0.2830214  0.29311675 0.28214954
 0.25042328 0.24714033 0.28472439 0.31261568 0.29138268 0.32421053
 0.31455592 0.28054524 0.29010637 0.31070192 0.29718427 0.30725662
 0.30572703 0.40971047 0.40532031 0.39884266 0.40197767 0.39685713
 0.39306993 0.39789167 0.39744059 0.3990502  0.40334521 0.40317471
 0.39899111 0.39995601 0.40170459 0.40163463 0.39692652 0.40036335
 0.40266812 0.4045743  0.3981236  0.39406701 0.3991057  0.39938949
 0.39744388 0.39932131 0.24822492 0.25376578 0.24429199 0.25953237
 0.24680859 0.24663193 0.25223524

  3%|█▎                                        | 301/10000 [04:01<2:11:28,  1.23it/s]


Step: 300
Delta_hat: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
b_hat: [3.47635207 2.12649513 5.80719217 7.18139949 6.09232965]
r_hat: [0.32970584 0.63900293 0.78343085 0.94010487 0.8364987 ]
beta_hat: [0.29108274 0.27841853 0.2833718  0.28387288 0.38205784 0.33650294
 0.33831105 0.2425098  0.30077588 0.3456322  0.27370792 0.26906138
 0.30799284 0.2912499  0.28892335 0.3099559  0.27054199 0.32549694
 0.32072336 0.28643878 0.28405104 0.29960833 0.28478079 0.30269991
 0.31594143 0.4049789  0.40421879 0.39542853 0.41136746 0.39493865
 0.39155025 0.4018371  0.39623305 0.39788556 0.4046658  0.40513154
 0.39756113 0.40055214 0.39886892 0.40221176 0.39969595 0.39798627
 0.40130774 0.4047734  0.39835377 0.39725205 0.40036531 0.40108668
 0.40092543 0.39706395 0.25303068 0.2508351  0.2462301  0.25842528
 0.2491546  0.24110451 0.25511596

  3%|█▎                                        | 314/10000 [04:14<2:10:58,  1.23it/s]


KeyboardInterrupt: 